# 十種猴子影像辨識

## 導入模組

In [246]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, Conv, MaxPool, onecold, onehotbatch, throttle, logitcrossentropy
using Augmentor
using Statistics
using Images
using JLD

## 載入資料

In [247]:
pl = Resize(128, 128) |>
     Either(1=>FlipX(), 1=>NoOp()) |>
     Either(1=>Zoom(0.9:0.1:1.2), 1=>NoOp()) |>
     CropSize(128, 128)

4-step Augmentor.ImmutablePipeline:
 1.) Resize to 128×128
 2.) Either: (50%) Flip the X axis. (50%) No operation.
 3.) Either: (50%) Zoom by I ∈ {0.9×0.9, 1.0×1.0, 1.1×1.1, 1.2×1.2}. (50%) No operation.
 4.) Crop a 128×128 window around the center

### 載入訓練資料

In [248]:
monkeys = Array{Float64}(undef, 1097, 128, 128, 3)
mon_key = 1
pbase = "training/training/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(128, 128, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:128)
                for k in (1:128)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        monkeys[mon_key, :, :, :] = c2
        mon_key += 1
    end
end

### 儲存訓練資料

In [249]:
save("monkey_data.jld", "data", monkeys)

### 讀入已儲存的訓練資料

In [250]:
train_x = load("monkey_data.jld")["data"]
train_x = reshape(train_x, 128, 128, 3, :)
train_x = Float32.(train_x)

128×128×3×1097 Array{Float32,4}:
[:, :, 1, 1] =
 0.470588   0.301961   0.913725  0.4        …  0.356863   0.0588235  0.321569
 0.219608   0.207843   0.780392  0.137255      0.443137   0.14902    0.407843
 0.435294   0.121569   0.427451  0.423529      0.278431   0.541176   0.231373
 0.996078   0.188235   0.521569  0.768627      0.537255   0.0705882  0.654902
 0.917647   0.603922   0.894118  0.505882      0.196078   0.254902   0.239216
 0.654902   0.407843   0.356863  0.439216   …  0.647059   0.223529   0.196078
 0.658824   0.192157   0.823529  0.784314      0.258824   0.486275   0.972549
 0.262745   0.0117647  0.866667  0.627451      0.396078   0.560784   0.184314
 0.443137   0.835294   0.298039  0.223529      0.129412   0.0862745  0.298039
 0.427451   0.270588   0.788235  0.909804      0.0745098  0.243137   0.117647
 0.192157   0.219608   0.321569  0.0588235  …  0.313726   0.231373   0.568627
 0.411765   0.498039   0.686275  0.152941      0.407843   0.356863   0.235294
 0.933333   0.53

### 載入驗證資料

In [251]:
val_monkeys = Array{Float64}(undef, 272, 128, 128, 3)
val_key = 1
pbase = "validation/validation/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(128, 128, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:128)
                for k in (1:128)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        val_monkeys[val_key, :, :, :] = c2
        val_key += 1
    end
end

### 儲存驗證資料

In [252]:
save("monkey_validation.jld", "data", val_monkeys)

### 讀入已儲存的驗證資料

In [253]:
test_x = load("monkey_validation.jld")["data"]
test_x = reshape(test_x, 128, 128, 3, :)
test_x = Float32.(test_x)

128×128×3×272 Array{Float32,4}:
[:, :, 1, 1] =
 0.435294   0.109804    0.8         …  0.396078   0.498039   0.0627451
 0.513726   0.976471    0.694118       0.211765   0.294118   0.0
 0.360784   0.388235    0.309804       0.294118   0.760784   0.254902
 0.121569   0.00392157  0.596078       0.239216   0.894118   0.25098
 0.2        0.258824    0.662745       0.419608   0.521569   0.490196
 0.0784314  0.25098     0.282353    …  0.333333   0.282353   0.317647
 0.823529   0.694118    0.752941       0.635294   0.866667   0.129412
 0.811765   0.117647    0.635294       0.258824   0.796078   0.262745
 0.607843   0.258824    0.262745       0.541176   0.447059   0.231373
 0.631373   0.423529    0.415686       0.580392   0.752941   0.533333
 0.490196   0.654902    0.12549     …  0.913725   0.505882   0.211765
 1.0        0.72549     0.176471       0.054902   0.196078   0.388235
 0.294118   0.243137    0.756863       0.290196   0.611765   0.145098
 ⋮                                  ⋱  ⋮        

### 訓練標籤

In [308]:
train_labels = Array{Int32}(undef, 1097)
key = 1
amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
for i in (0:9)
    for j in (1:amount[i + 1])
        train_labels[key] = i
        key += 1
    end
end
train_y = onehotbatch(train_labels, 0:9)

10×1097 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

### 驗證標籤

In [313]:
val_labels = Array{Int32}(undef, 272)
key = 1
amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
for i in (0:9)
    for j in (1:amount[i + 1])
        val_labels[key] = i
        key += 1
    end
end
test_y = onehotbatch(val_labels, 0:9)

10×272 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [255]:
batchsize = 32
train = DataLoader(train_x, train_y, batchsize = batchsize, shuffle = true)
test = DataLoader(test_x, test_y, batchsize = batchsize)

DataLoader((Float32[0.43529412 0.10980392 … 0.49803922 0.0627451; 0.5137255 0.9764706 … 0.29411766 0.0; … ; 0.5529412 0.30588236 … 0.70980394 0.34117648; 0.77254903 0.72156864 … 0.26666668 0.19215687]

Float32[0.75686276 0.09019608 … 0.06666667 0.8509804; 0.3137255 0.4392157 … 0.23529412 0.47058824; … ; 0.27450982 0.29803923 … 0.4392157 0.34117648; 0.46666667 0.5764706 … 0.61960787 0.39607844]

Float32[0.6862745 0.93333334 … 0.16862746 0.7137255; 0.25882354 0.8862745 … 0.0 0.43529412; … ; 0.11372549 0.21176471 … 0.23529412 0.54901963; 0.61960787 0.19215687 … 0.22352941 0.40784314]

Float32[0.22352941 0.19607843 … 0.31764707 0.8117647; 0.2901961 0.42745098 … 0.42352942 0.47058824; … ; 0.27450982 0.6431373 … 0.44705883 0.14901961; 0.5058824 0.28235295 … 0.54509807 0.7372549]

Float32[0.79607844 0.34509805 … 0.69411767 0.20392157; 0.99215686 0.54901963 … 0.11372549 0.49803922; … ; 0.19607843 0.9254902 … 0.6156863 0.24705882; 0.34117648 0.03137255 … 0.40784314 0.48235294]

Float32[0.137254

## CNN 模型

In [133]:
'''
model = Chain(
    
    # First Block
    Conv((3, 3), pad = (1, 1), 3 => 64, relu), # 128x128
    Dropout(0.3f0),
    Conv((3, 3), pad = (1, 1), 64 => 64, relu), # 128x128
    MaxPool((2, 2), stride = (2, 2)), # 64x64
    
    # Second Block
    Conv((3, 3), 64 => 128, pad = (1, 1), relu), # 64x64
    Dropout(0.4f0),
    Conv((3, 3), 128 => 128, pad = (1, 1), relu), # 64x64
    MaxPool((2, 2), stride = (2, 2)), # 32x32
    
    # Third Block
    Conv((3, 3), 128 => 256, pad = (1, 1), relu), # 32x32
    Dropout(0.4f0),
    Conv((3, 3), 256 => 256, pad = (1, 1), relu), # 32x32
    Dropout(0.4f0),
    Conv((3, 3), 256 => 256, pad = (1, 1), relu), # 32x32
    MaxPool((2, 2), stride = (2, 2)), # 16x16
    
    # Forth Block
    Conv((3, 3), 256 => 512, pad = (1, 1), relu), # 16x16
    Dropout(0.4f0),
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 16x16
    Dropout(0.4f0),
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 16x16
    MaxPool((2, 2), stride = (2, 2)), # 8x8
    
    # Fifth Block
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 8x8
    Dropout(0.4f0),
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 8x8
    Dropout(0.4f0),
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 8x8
    MaxPool((2, 2), stride = (2, 2)), # 4x4
    
    # Flatten & Dense
    flatten,
    Dense(8192, 1024),
    Dropout(0.5f0),
    Dense(1024, 256),
    Dropout(0.5f0),
    Dense(256, 10))
'''

Chain(Conv((3, 3), 3=>64, relu), Dropout(0.3), Conv((3, 3), 64=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 64=>128, relu), Dropout(0.4), Conv((3, 3), 128=>128, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 128=>256, relu), Dropout(0.4), Conv((3, 3), 256=>256, relu), Dropout(0.4), Conv((3, 3), 256=>256, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 256=>512, relu), Dropout(0.4), Conv((3, 3), 512=>512, relu), Dropout(0.4), Conv((3, 3), 512=>512, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 512=>512, relu), Dropout(0.4), Conv((3, 3), 512=>512, relu), Dropout(0.4), Conv((3, 3), 512=>512, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(8192, 1024), Dropout(0.5), Dense(1024, 256), Dropout(0.5), Dense(256, 10))

In [412]:
model_easy = Chain(
    
    BatchNorm(3),
    Conv((3, 3), 3 => 32, pad = (1, 1), relu), # 128x128
    MaxPool((2, 2), stride = (2, 2)), # 64x64
    Conv((3, 3), 32 => 64, pad = (1, 1), relu), # 64x64 

    BatchNorm(64),
    Conv((3, 3), 64 => 128, pad = (1, 1), relu), # 64x64
    MaxPool((2, 2), stride = (2, 2)), # 32x32
    Conv((3, 3), 128 => 128, pad = (1, 1), relu), # 32x32

    BatchNorm(128),
    Conv((3, 3), 128 => 64, pad = (1, 1), relu), # 32x32
    MaxPool((2, 2), stride = (2, 2)), # 16x16
    Conv((3, 3), 64 => 32, pad = (1, 1), relu), # 16x16

    BatchNorm(32),
    Conv((3, 3), 32 => 16, pad = (1, 1), relu), # 16x16
    MaxPool((2, 2), stride = (2, 2)), # 8x8
    
    Flux.flatten,
    Dense(1024, 128, relu),
    BatchNorm(128),
    Dense(128, 10),
    softmax
)

Chain(BatchNorm(3), Conv((3, 3), 3=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), BatchNorm(64), Conv((3, 3), 64=>128, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 128=>128, relu), BatchNorm(128), Conv((3, 3), 128=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 64=>32, relu), BatchNorm(32), Conv((3, 3), 32=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(1024, 128, relu), BatchNorm(128), Dense(128, 10), softmax)

In [413]:
size(train_x)

(128, 128, 3, 1097)

In [414]:
model_easy(train_x[:, :, :, 1:32])

10×32 Array{Float32,2}:
 0.101952   0.101541   0.102902   …  0.101779   0.102188   0.102393
 0.104846   0.103189   0.103687      0.104121   0.104048   0.10383
 0.0880593  0.088655   0.0886757     0.0887961  0.0885093  0.0880557
 0.0987595  0.0988902  0.0984387     0.0991215  0.09829    0.0987322
 0.10183    0.10185    0.101333      0.101009   0.101206   0.100863
 0.0958818  0.0961527  0.0959432  …  0.0956733  0.0962364  0.0962849
 0.104368   0.104277   0.104079      0.103947   0.103692   0.104561
 0.10219    0.103157   0.102895      0.102698   0.103211   0.102559
 0.100953   0.100953   0.101651      0.101587   0.101653   0.100993
 0.10116    0.101335   0.100394      0.101268   0.100967   0.101728

## 損失函數

In [415]:
loss(x, y) = logitcrossentropy(model_easy(x), y)

loss (generic function with 1 method)

In [416]:
accuracy(x, y, model) = mean(onecold(model_easy(x)) .== onecold(y))

accuracy (generic function with 1 method)

## Callback 函式

In [417]:
function callback()
    print("Testing ")
    @show loss(test_x, test_y)
    print("Testing ")
    @show accuracy(test_x, test_y, model_easy)
    println()
    accuracy(test_x, test_y, model_easy) > 0.8 && Flux.stop()
end

callback (generic function with 1 method)

## 模型訓練

In [418]:
epochs = 15
@epochs epochs Flux.train!(loss, params(model_easy), train, 
                           ADAM(0.005, (0.9, 0.8)), 
                           cb = throttle(callback, 500))

┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.3026733f0
Testing accuracy(test_x, test_y, model_easy) = 0.09926470588235294



┌ Info: Epoch 2
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.2963178f0
Testing accuracy(test_x, test_y, model_easy) = 0.09926470588235294



┌ Info: Epoch 3
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

Testing loss(test_x, test_y) = 2.3117297f0
Testing accuracy(test_x, test_y, model_easy) = 0.1323529411764706



┌ Info: Epoch 14
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.3086317f0
Testing accuracy(test_x, test_y, model_easy) = 0.13602941176470587



┌ Info: Epoch 15
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.2771828f0
Testing accuracy(test_x, test_y, model_easy) = 0.15808823529411764



## 模型評估

In [445]:
accuracy(test_x, test_y, model_easy)

0.1875